In [1]:
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset,DataLoader
import pandas as pd
from sklearn import preprocessing

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
!wget https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv

--2021-05-16 01:34:24--  https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3858 (3.8K) [text/plain]
Saving to: ‘iris.csv.5’

iris.csv.5          100%[===================>]   3.77K  --.-KB/s    in 0.001s  

2021-05-16 01:34:24 (3.22 MB/s) - ‘iris.csv.5’ saved [3858/3858]



### Dataloader

In [4]:
class IrisDataset(Dataset):
    def __init__(self, file):
        self.data = pd.read_csv(file)
        input_features = self.data.values[:, 0:4].astype('float32')
        self.x_train = torch.from_numpy(input_features)
        
        label_encoder = preprocessing.LabelEncoder()
        self.data['species'] = label_encoder.fit_transform(self.data['species'])
        mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
        print(mapping)
        target_label = self.data['species'].values
        
        self.y_train = torch.from_numpy(target_label)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        return (self.x_train[idx],self.y_train[idx])

In [5]:
dataset = IrisDataset('iris.csv')
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

{'setosa': 0, 'versicolor': 1, 'virginica': 2}


In [7]:
train_features[0],train_labels[0]

(tensor([5.3000, 3.7000, 1.5000, 0.2000]), tensor(0))

### Model

In [6]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [7]:
model = Model(4).to(device)

### Optimizer and loss

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train

In [9]:
epochs = 20

for epoch in range(1,epochs+1):
    print("EPOCH:",epoch,end=" ")
    running_loss=0
    running_acc=0
    
    for data,labels in dataloader:
        data,labels=data.to(device),labels.to(device)
        optimizer.zero_grad()
        output=model.forward(data)
        loss=criterion(output,labels)
        
        result=torch.argmax(output,dim=1)
        running_loss+=loss.item()
        running_acc+=torch.mean((result==labels).type(torch.float))
        
        loss.backward()
        optimizer.step()
    else:
        train_loss=running_loss/len(dataloader)
        train_acc=running_acc/len(dataloader)
        
        print("Training Loss: {:.3f}".format(train_loss),end=" ") 
       
        print("Train Accuracy: {:.2f}%".format(train_acc.item()*100))
        

EPOCH: 1 Training Loss: 1.083 Train Accuracy: 34.38%
EPOCH: 2 Training Loss: 1.068 Train Accuracy: 32.29%
EPOCH: 3 Training Loss: 1.041 Train Accuracy: 33.33%
EPOCH: 4 Training Loss: 1.016 Train Accuracy: 60.62%
EPOCH: 5 Training Loss: 0.982 Train Accuracy: 65.62%
EPOCH: 6 Training Loss: 0.943 Train Accuracy: 65.62%
EPOCH: 7 Training Loss: 0.893 Train Accuracy: 67.71%
EPOCH: 8 Training Loss: 0.858 Train Accuracy: 66.67%
EPOCH: 9 Training Loss: 0.840 Train Accuracy: 64.17%
EPOCH: 10 Training Loss: 0.806 Train Accuracy: 90.62%
EPOCH: 11 Training Loss: 0.783 Train Accuracy: 88.13%
EPOCH: 12 Training Loss: 0.758 Train Accuracy: 91.87%
EPOCH: 13 Training Loss: 0.746 Train Accuracy: 91.25%
EPOCH: 14 Training Loss: 0.729 Train Accuracy: 93.13%
EPOCH: 15 Training Loss: 0.715 Train Accuracy: 96.46%
EPOCH: 16 Training Loss: 0.701 Train Accuracy: 96.25%
EPOCH: 17 Training Loss: 0.697 Train Accuracy: 92.50%
EPOCH: 18 Training Loss: 0.696 Train Accuracy: 91.46%
EPOCH: 19 Training Loss: 0.707 Train 

### Test

In [56]:
train_features, train_labels = next(iter(dataloader))

In [57]:
train_features[8],train_labels[8]

(tensor([4.8000, 3.0000, 1.4000, 0.3000]), tensor(0))

In [60]:
data = np.array([[4.8000, 3.0000, 1.4000, 0.3000]], dtype=np.float32)
data = torch.from_numpy(data).to(device)

In [62]:
torch.argmax(model(data),dim=1)

tensor([0], device='cuda:0')